#### Task 1: Simple vector embedding generation

**Objective:**
Generate vector embeddings from text data.

**Task Description:**

- load huggingface embedding model (`model_name="sentence-transformers/all-mpnet-base-v2"`)
- embed simple text queries

How to select the right embedding model: [MTEB - Massive Text Embedding Benchmark](https://huggingface.co/blog/mteb)

**Useful links:**

- [Langchain Chroma](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/)


In [12]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

# ADD HERE YOUR CODE
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-mpnet-base-v2")

In [13]:
text = "This is a test document."

# ADD HERE YOUR CODE
# Perform vector search
query_vector = embedding_model.embed_query(text)

print(f"Embedding vector length: {len(query_vector)}")
print(query_vector[:10])

Embedding vector length: 768
[-0.04895179718732834, -0.03986203297972679, -0.021562788635492325, 0.00990856159478426, -0.038103923201560974, 0.01268438808619976, 0.04349448159337044, 0.07183388620615005, 0.009748609736561775, -0.006987091153860092]


#### Task 2: Generate embedding vectors with custom dataset

**Objective:**
Load custom dataset, preprocess it and generate vector embeddings.

**Task Description:**

- load pdf document "AI_Book.pdf" via langchain document loader: `PyPDFLoader`
- use RecursiveCharacterTextSplitter to split documents into chunks
- generate embeddings for single documents

**RecursiveCharacterTextSplitter:**

This text splitter is the recommended one for generic text. It is parameterized by a list of characters. It tries to split on them in order until the chunks are small enough. The default list is `["\n\n", "\n", " ", ""]`. This has the effect of trying to keep all paragraphs (and then sentences, and then words) together as long as possible, as those would generically seem to be the strongest semantically related pieces of text.

**Useful links:**

- [Langchain PyPDFLoader](https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html)
- [Langchain RecursiveCharacterTextSplitter](https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html)


In [14]:
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
import re

pdf_doc = "./AI_Book.pdf"

# Create pdf data loader
# ADD HERE YOUR CODE
loader = PyPDFLoader(file_path=pdf_doc)

# Load and split documents in chunks
# ADD HERE YOUR CODE

pages = loader.load()
pages_chunked = RecursiveCharacterTextSplitter().split_documents(pages)

# Function to clean text by removing invalid unicode characters, including surrogate pairs
def clean_text(chunk):
    # Remove surrogate pairs
    text = chunk.page_content
    text = re.sub(r'[\ud800-\udfff]', '', text)
    # Optionally remove non-ASCII characters (depends on your use case)
    text = re.sub(r'[^\x00-\x7F]+', '', text)
    return Document(page_content=text, metadata=chunk.metadata)

pages_chunked_cleaned = [clean_text(chunk) for chunk in pages_chunked]

print(pages_chunked[0])

page_content='Aurélien GéronHands-on Machine Learning with
Scikit-Learn, Keras, and
TensorFlow
Concepts, Tools, and Techniques to
Build Intelligent SystemsSECOND EDITION
Boston Farnham Sebastopol Tokyo Beijing Boston Farnham Sebastopol Tokyo Beijing' metadata={'source': './AI_Book.pdf', 'page': 2}


In [15]:
print(pages_chunked[1])

page_content='978-1-492-03264-9
[LSI]Hands-on Machine Learning with Scikit-Learn, Keras, and TensorFlow
by Aurélien Géron
Copyright © 2019 Aurélien Géron. All rights reserved.
Printed in the United States of America.
Published by O’Reilly Media, Inc., 1005 Gravenstein Highway North, Sebastopol, CA 95472.
O’Reilly books may be purchased for educational, business, or sales promotional use. Online editions are
also available for most titles ( http://oreilly.com ). For more information, contact our corporate/institutional
sales department: 800-998-9938 or corporate@oreilly.com .
Editor:  Nicole Tache
Interior Designer:  David FutatoCover Designer:  Karen Montgomery
Illustrator:  Rebecca Demarest
June 2019:  Second Edition
Revision History for the Early Release
2018-11-05: First Release
2019-01-24: Second Release
2019-03-07: Third Release
2019-03-29: Fourth Release
2019-04-22: Fifth Release
See http://oreilly.com/catalog/errata.csp?isbn=9781492032649  for release details.
The O’Reilly logo 

In [16]:
print(f"Number of text chunks: {len(pages_chunked)}")

Number of text chunks: 507


#### Task 3: Store vector embeddings from pdf document to ChromaDB vector database.

**Objective:**

Store vector embeddings into ChromaDB to store knowledge.

**Task Description:**

- create chromadb client
- create chromadb collection
- create langchain chroma db client
- store text document chunks and vector embeddings to vector databases

**Useful links:**

- [Langchain How To](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#initialization-from-client)


In [17]:
from langchain_chroma import Chroma
import chromadb
import chromadb
from chromadb.config import DEFAULT_TENANT, DEFAULT_DATABASE, Settings

client = chromadb.HttpClient(
    host="localhost",
    port=8000,
    ssl=False,
    headers=None,
    settings=Settings(allow_reset=True, anonymized_telemetry=False),
    tenant=DEFAULT_TENANT,
    database=DEFAULT_DATABASE,
)

# ADD HERE YOUR CODE
# Create a collection
collection = client.get_or_create_collection("test_collection")
#embedding_model = "llama3.2"

# ADD HERE YOUR CODE
# Create chromadb
vector_db_from_client = Chroma(
    client=client,
    collection_name="test_collection",
    embedding_function=embedding_model,
)

In [18]:
from uuid import uuid4

uuids = [str(uuid4()) for _ in range(len(pages_chunked[:50]))]

# ADD HERE YOUR CODE
vector_db_from_client.add_documents(documents=pages_chunked_cleaned[:50], id=uuids)
#vector_db_from_client.add_texts(pages_chunked_cleaned, id = uuids)

['34c0faa6-4b9f-473b-9332-4cc347509f4f',
 'a94748b8-2645-4f0a-adbb-f70b7a51cc90',
 '1daefd95-eaaa-4e13-8593-945d3cac8dca',
 'cab8ae14-6e4f-40c6-b911-75f378ffd1a0',
 'e5cd02fe-3bac-4f9c-bd40-c0f7d955907b',
 '017ce799-ac14-4ba6-a836-982bb63b2f2e',
 '119f1821-606a-4330-a54a-3238b82c078b',
 'b6ebecda-7985-4e56-a931-7a55c64d2496',
 '65728310-f73c-4f43-a163-be7c9b7bbbc4',
 'e3909565-6d85-4491-90d4-f332d8ab7335',
 '524d14b7-5ee4-4299-a459-f0627e5467c0',
 'f9e7a225-736b-499b-9af0-e3be072e20bb',
 'fc38f1e9-2b37-4698-bf68-ab7a16977a4c',
 '7b5fba41-6c54-4c12-818d-6238fafb3bd9',
 '5b978c51-b7ce-406d-8e83-8f4b9c8d2be4',
 '4fd2f720-149c-4714-870b-0210eeaf9e7d',
 '1eefa1fc-a3bf-46a8-a19a-50f7889753e6',
 '2f17d31b-dcba-4341-920f-700063c0a715',
 'd3ef51fe-ae37-4f4b-92c7-3fdd89ae9edb',
 '51e4c94c-3223-401c-aef2-95bb91c339d5',
 'a503ff13-dc40-4809-b0df-19d3a114f92c',
 '70e5bc64-8f57-45b0-8ed2-5a0a1112a294',
 'b86e42d3-f6c8-4b17-b33a-2b22fbc0490a',
 'b2c6deab-7da8-4140-b00a-16384172bb52',
 '0d2acd1a-06e5-

In [19]:
client.count_collections()

1

In [21]:
# client.delete_collection("ai_model_book")

#### Task 4: Access ChromaDB and perform vector search

**Objective:**

Use query to perform vector search against ChromaDB vector database

**Task Description:**

- define query
- run vector search
- print k=3 most similar documents

**Useful links:**

- [Langchain Query ChromaDB](https://python.langchain.com/v0.2/docs/integrations/vectorstores/chroma/#query-directly)


In [22]:
search_query = "Types of Machine Learning Systems"

results = vector_db_from_client.similarity_search(search_query, k=3)

for res in results:
    print(res.page_content)
    print(res.metadata)
    print("\n")

Types of Machine Learning Systems
There are so many different types of Machine Learning systems that it is useful to
classify them in broad categories based on:
Whether or not they are trained with human supervision (supervised, unsuper
vised, semisupervised, and Reinforcement Learning)
Whether or not they can learn incrementally on the fly (online versus batch
learning)
Whether they work by simply comparing new data points to known data points,
or instead detect patterns in the training data and build a predictive model, much
like scientists do (instance-based versus model-based learning)
These criteria are not exclusive; you can combine them in any way you like. For
example, a state-of-the-art spam filter may learn on the fly using a deep neural net
work model trained using examples of spam and ham; this makes it an online, model-
based, supervised learning system.
Lets look at each of these criteria a bit more closely.
Supervised/Unsupervised Learning
Machine Learning systems can be